## Drive Mount

In [1]:
base_path = "/content/drive/My Drive/yl_tez"

In [2]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Mounted at /content/drive


In [3]:
!pip install datasets
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [4]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

# Değişkenler

In [5]:
model_name = "ytu-ce-cosmos/turkish-gpt2-medium"
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)
veri_seti_adi =  "ana_veri_seti_deneme.csv"
data_filepath = os.path.join(base_path, veri_seti_adi)
new_special_tokens = ["<USER>", "<ASSISTANT>", "<DOCTOR_TITLE>", "<SPECIALITY>", "</s>"]

# Veri setini oku

In [6]:
# Veri dosyasını oku
df = pd.read_csv(data_filepath)

In [7]:
print(df.head().to_markdown())
len(df)

|    | text                                                                                                                                                                                                                                                                                                                                         |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Durum: Ben hipertansiyon hastası değilim.  Bende kalp hastalığı yok.  Benim kilom biraz fazla.  Ben yaşlı bir insanım.  Ben bir kadınım.  Ben hayatım boyunca asla sigara içmedim.  Ben HbA1c seviyeme göre prediyabet durumundayım.  Benim kan şekerim aşırı derecede yüksek.  Bilgi: Sende diyabet hastalığı mevcut

1000

In [8]:
dataset = Dataset.from_pandas(df)
train_test_split = dataset.train_test_split(test_size=0.1)

train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [9]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 1000
})


# Tokenizer Güncelleme

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"additional_special_tokens": new_special_tokens})
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# Model Eğitme

In [11]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/946 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Embedding(50259, 1024)

In [15]:
training_args = SFTConfig(
    output_dir=model_yolu,  # Modelin kaydedileceği dizin
    report_to="none",  # Raporlama aracı
    dataloader_drop_last=True,  # Son batch'i bırak
    per_device_eval_batch_size=32,  # Değerlendirme batch boyutu
    per_device_train_batch_size=32, # Her cihaz başına eğitim batch boyutu
    save_total_limit=3,  # En fazla 3 kontrol noktası sakla,,
    save_steps=10,  # Kaydetme adımı,
    eval_steps=100,  # Test etme adımı
    eval_strategy="steps",  # Değerlendirme stratejisi adım bazında
    logging_steps=10,  # Her 50 adımda bir loglama yap
    num_train_epochs=5,  # Eğitim epoch sayısı,
    gradient_checkpointing=False,  # Gradient checkpointing
    gradient_accumulation_steps=4,  # Gradient accumulation steps,
    fp16=False,  # 16-bit float training
)

In [16]:
trainer = SFTTrainer(
        model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        max_seq_length=512, # context length
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [17]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=35, training_loss=0.4541271209716797, metrics={'train_runtime': 353.8075, 'train_samples_per_second': 14.132, 'train_steps_per_second': 0.099, 'total_flos': 661641201057792.0, 'train_loss': 0.4541271209716797, 'epoch': 4.516129032258064})

Modeli Kaydet

In [94]:
trainer.save_model()

Logları kaydet

In [95]:
import json
# Eğitim ve değerlendirme metriklerini kaydetme
log_history = trainer.state.log_history
log_file = os.path.join(model_yolu, "training_log.json")
with open(log_file, "w") as f:
    json.dump(log_history, f)